In [78]:
from collections import Counter

def calculate_sparse_and_topN_accuracy(dense_result, mrc, are_texts_similar, topk=[1,2,3,5,10,15,20]):
    # 각 top N에 정답이 몇 개 있는지 카운팅할 Counter
    correct_topN_counter = Counter()

    # 전체 row 수
    total_rows = len(dense_result)
    total_cols = len(dense_result.columns)

    # dense_result의 모든 row에 대해 반복
    for idx, row in dense_result.iterrows():
        mrc_id = row['id']  # 각 row의 id 추출

        # mrc에서 해당 id의 context 추출
        try:
            mrc_context = mrc[mrc['id'] == mrc_id]['context'].values[0]
        except IndexError:
            print(f"Context not found for ID: {mrc_id}")
            continue  # 만약 mrc에서 id가 없으면 다음 row로 넘어감

        # topN_context들 중에서 mrc_context와 일치하는 index를 찾음
        for i in range(1, total_cols-1):  # top1_context부터 top100_context까지 확인
            top_context = row[f'top{i}_context']

            # 공백 및 줄바꿈 처리
            mrc_context = mrc_context.strip().replace("\n\n", " ").replace("\n", " ")
            top_context = top_context.strip().replace("\n\n", " ").replace("\n", " ")

            # 유사도 비교
            if are_texts_similar(mrc_context, top_context):
                correct_topN_counter[i] += 1  # 해당하는 topN에 카운트 추가
                break

    # Sparse accuracy 계산 및 출력
    sparse_accuracy = (sum(correct_topN_counter.values()) / total_rows) * 100
    print(f"Total Count: {total_rows}")
    print(f"Missing Count: {total_rows - sum(correct_topN_counter.values())}")
    print(f"Top 100 Sparse accuracy: {sparse_accuracy:.2f}%")

    # 각 topN에 대한 정확도 출력
    print("\nAccuracy Results:")
    for top_n in topk:  # top1부터 top5까지 정확도 확인
        correct_count = sum([count for n, count in correct_topN_counter.items() if n <= top_n])
        accuracy = (correct_count / total_rows) * 100
        print(f"Top {top_n} accuracy: {accuracy:.2f}% ({correct_count}/{total_rows})")

In [66]:
from difflib import SequenceMatcher

def are_texts_similar(text1, text2, threshold=0.95, print_score=False):
    """
    두 텍스트가 주어진 유사도 threshold 이상으로 유사한지 확인하는 함수.
    
    Parameters:
        text1 (str): 첫 번째 텍스트
        text2 (str): 두 번째 텍스트
        threshold (float): 유사도를 판단하는 기준 (0 ~ 1 사이 값, 기본값은 0.9)

    Returns:
        bool: 유사도가 threshold 이상이면 True, 그렇지 않으면 False
    """
    
    # 불필요한 공백, 줄바꿈 등을 제거
    text1_clean = text1.strip().replace("\n\n", " ").replace("\n", " ")
    text2_clean = text2.strip().replace("\n\n", " ").replace("\n", " ")

    # 유사도 계산 함수
    def similarity(a, b):
        return SequenceMatcher(None, a, b).ratio()

    # 유사도 계산
    similarity_score = similarity(text1_clean, text2_clean)
    if print_score:
        print(f"Similarity: {similarity_score * 100:.2f}%")

    # 유사도가 threshold 이상이면 True 반환
    return similarity_score >= threshold

In [35]:
import json
import pandas as pd
from datasets import load_from_disk


data = json.load(open('../data/raw/wikipedia_documents.json'))
wiki = pd.DataFrame(data).T

dataset = load_from_disk("../data/raw/train_dataset/")
train_df = pd.DataFrame(dataset['train'])
valid_df = pd.DataFrame(dataset['validation'])

mrc = pd.concat([train_df, valid_df])

In [84]:
import json
import pandas as pd

BGEm3ko_max = pd.read_csv('../data/pipeline/BM25Ensemble_top100_BGEm3ko_max.csv')
gteMultibase = pd.read_csv('../data/pipeline/BM25Ensemble_top100_gteMultibase_max.csv')
KoE5_max = pd.read_csv('../data/pipeline/BM25Ensemble_top100_KoE5_max.csv')
KoE5_mean = pd.read_csv('../data/pipeline/BM25Ensemble_top100_KoE5_mean.csv')
original = pd.read_csv('../data/pipeline/BM25Ensemble_top100_original.csv')
ParaMiniLM_max = pd.read_csv('../data/pipeline/BM25Ensemble_top100_ParaMiniLM_max.csv')
# qwen_max = pd.read_csv('../data/pipeline/BM25Ensemble_top100_Qwen_max.csv')
upskyyBGEm3korean_max = pd.read_csv('../data/pipeline/BM25Ensemble_top100_upskyyBGEm3korean_max.csv')
qwen2_1_5b_max = pd.read_csv('../data/pipeline/BM25Ensemble_top100_Qwen2-1_5b_max.csv')

In [85]:
topk = [1,2,3,5,10,15,20]
calculate_sparse_and_topN_accuracy(qwen2_1_5b_max, mrc, are_texts_similar, topk=topk)
print('*'*50)
# calculate_sparse_and_topN_accuracy(gteMultibase, mrc, are_texts_similar, topk=topk)
# print('*'*50)
# calculate_sparse_and_topN_accuracy(ParaMiniLM_max, mrc, are_texts_similar, topk=topk)
# print('*'*50)
# calculate_sparse_and_topN_accuracy(original, mrc, are_texts_similar, topk=topk)
# print('*'*50)
# calculate_sparse_and_topN_accuracy(upskyyBGEm3korean_max, mrc, are_texts_similar, topk=topk)
# print('*'*50)
# calculate_sparse_and_topN_accuracy(BGEm3ko_max, mrc, are_texts_similar, topk=topk)
# print('*'*50)
# calculate_sparse_and_topN_accuracy(KoE5_max, mrc, are_texts_similar, topk=topk)
# print('*'*50)
# calculate_sparse_and_topN_accuracy(KoE5_mean, mrc, are_texts_similar, topk=topk)


Total Count: 240
Missing Count: 5
Top 100 Sparse accuracy: 97.92%

Accuracy Results:
Top 1 accuracy: 57.08% (137/240)
Top 2 accuracy: 67.08% (161/240)
Top 3 accuracy: 73.75% (177/240)
Top 5 accuracy: 80.83% (194/240)
Top 10 accuracy: 85.42% (205/240)
Top 15 accuracy: 88.33% (212/240)
Top 20 accuracy: 90.83% (218/240)
**************************************************


In [92]:
import pandas as pd
from collections import Counter

# 여러 dataframe을 받아 hard voting 수행
def hard_voting(df_list):
    result_rows = []  # 결과를 저장할 리스트
    
    # 각 query마다 처리
    for idx in range(len(df_list[0])):
        id_val = df_list[0].iloc[idx]['id']
        question_val = df_list[0].iloc[idx]['question']
        chosen_contexts = []  # 각 query에 대해 선택된 topn_context를 저장할 리스트

        # top1_context부터 top100_context까지 처리
        for top_n in range(1, 101):
            candidates = []

            # 모든 dataframe에서 해당 query의 topn_context 후보 수집
            for df in df_list:
                candidates.append(df.iloc[idx][f'top{top_n}_context'])

            # 다수결로 가장 많이 나온 후보 선택 (중복된 문서 제외)
            filtered_candidates = [c for c in candidates if c not in chosen_contexts]
            if len(filtered_candidates) == 0:
                chosen_context = None
            else:
                chosen_context = Counter(filtered_candidates).most_common(1)[0][0]

            chosen_contexts.append(chosen_context)

        # 결과에 저장
        result_rows.append([id_val, question_val] + chosen_contexts)
    
    # 리스트로 모은 결과를 DataFrame으로 변환
    result = pd.DataFrame(result_rows, columns=df_list[0].columns)
    
    return result

# hard voting 실행
# df_list = [BGEm3ko_max, gteMultibase, KoE5_max, KoE5_mean, original, ParaMiniLM_max, upskyyBGEm3korean_max]
df_list = [BGEm3ko_max, KoE5_max, original]
final_df = hard_voting(df_list)

x = 20
final_df_cut = final_df[['id', 'question'] + [f'top{i}_context' for i in range(1, x+1)]]
# check is null
print(final_df_cut.isnull().sum().sum())
calculate_sparse_and_topN_accuracy(final_df_cut, mrc, are_texts_similar, topk=topk)

5
Total Count: 240
Missing Count: 8
Top 100 Sparse accuracy: 96.67%

Accuracy Results:
Top 1 accuracy: 72.92% (175/240)
Top 2 accuracy: 82.92% (199/240)
Top 3 accuracy: 86.67% (208/240)
Top 5 accuracy: 90.00% (216/240)
Top 10 accuracy: 95.00% (228/240)
Top 15 accuracy: 96.25% (231/240)
Top 20 accuracy: 96.67% (232/240)


In [121]:
import pandas as pd
from collections import Counter

# 여러 dataframe을 받아 hard voting 수행
def hard_voting(df_list):
    result_rows = []  # 결과를 저장할 리스트

    # 각 query마다 처리
    for idx in range(len(df_list[0])):
        id_val = df_list[0].iloc[idx]['id']
        question_val = df_list[0].iloc[idx]['question']
        chosen_contexts = []  # 각 query에 대해 선택된 topn_context를 저장할 리스트

        top_n = 1  # top_n의 초기값 설정
        while top_n <= 100:  # top_n이 100 이하일 때까지 반복
            candidates = []

            # 모든 dataframe에서 해당 query의 topn_context 후보 수집
            for df in df_list:
                candidates.append(df.iloc[idx][f'top{top_n}_context'])

            # 다수결로 가장 많이 나온 후보 선택 (중복된 문서 제외)
            filtered_candidates = [c for c in candidates if c not in chosen_contexts]

            if len(filtered_candidates) > 0:
                chosen_context = Counter(filtered_candidates).most_common(1)[0][0]
                chosen_contexts.append(chosen_context)
                top_n += 1  # 다음 top_n으로 이동
            else:
                top_n += 1  # 후보가 없으면 top_n만 증가하고 다시 반복

            # 후보가 없을 때 (None 추가)
            if top_n > 100:
                chosen_contexts.append(None)
                break

        # 결과에 저장
        result_rows.append([id_val, question_val] + chosen_contexts[:100])

    # 리스트로 모은 결과를 DataFrame으로 변환
    result = pd.DataFrame(result_rows, columns=df_list[0].columns)
    
    return result

# hard voting 실행
# df_list = [BGEm3ko_max, gteMultibase, KoE5_max, KoE5_mean, original, ParaMiniLM_max, upskyyBGEm3korean_max]
'''
original, BGEm3ko_max, (KoE5_max, KoE5_mean)

original, BGEm3ko_max
original, KoE5_max
original, KoE5_mean
BGEm3ko_max, KoE5_max
BGEm3ko_max, KoE5_mean
original, BGEm3ko_max, KoE5_max
original, BGEm3ko_max, KoE5_mean
'''

df_list = [original, BGEm3ko_max, KoE5_mean]
final_df = hard_voting(df_list)

x = 20
final_df_cut = final_df[['id', 'question'] + [f'top{i}_context' for i in range(1, x+1)]]
# check is null
print(f'Null sum(): {final_df_cut.isnull().sum().sum()}')
# topk = [i for i in range(1, 21)]
topk = [1,2,3,5,10,15,20]
calculate_sparse_and_topN_accuracy(final_df_cut, mrc, are_texts_similar, topk=topk)


Null sum(): 0
Total Count: 240
Missing Count: 20
Top 100 Sparse accuracy: 91.67%

Accuracy Results:
Top 1 accuracy: 72.50% (174/240)
Top 2 accuracy: 82.50% (198/240)
Top 3 accuracy: 85.42% (205/240)
Top 5 accuracy: 86.67% (208/240)
Top 10 accuracy: 89.58% (215/240)
Top 15 accuracy: 91.25% (219/240)
Top 20 accuracy: 91.67% (220/240)


In [105]:
final_df

,id,question,top1_context,top2_context,top3_context,top4_context,top5_context,top6_context,top7_context,top8_context,...,top91_context,top92_context,top93_context,top94_context,top95_context,top96_context,top97_context,top98_context,top99_context,top100_context
0,mrc-0-003264,처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,2008년 11월에는 금융감독원 내에 금융위원회와 합동으로 ‘기업재무개선지원단’을 ...,1884년 5월 4일 테네시주의 기차의 1등석에 앉아 있던 웰스를 향해 승무원이 열...,"순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...","세계 1차 대전이 일어나자, 메테스키는 미국 해병대에 지원해 전기 전공자로 상하이의...",2012년 8월 17일 미국 증권거래위원회는 폴 벅스와 노스 캐롤라이나 주의 지크 ...,룰루레몬은 1998년 캐나다 브리티시컬럼비아주 밴쿠버에서 칩 윌슨(Chip Wils...,1939년 일본 입명관 대학 법과를 졸업하고 1940년 일본고등문관 사법과와 만주 ...,서울민사지방법원 22단독 판사로 재직하던 1980년 9월 24일에 자기앞 수표 발행...,...,메리 루이스 부스(Mary Louise Booth)는 현재의 애팽크인 Millvil...,1940년 8월 8일 경상남도 통영시에서 태어나 부산고등학교와 서울대학교(58학번)...,1934년 전라북도 진안군에서 태어나 전주고등학교와 전북대학교 정치학과를 나왔다. ...,"날짜=2017-12-06\n \n \n집행유예(執行猶豫, probation)는 형법...","전남 목포(木浦)사람으로, 1905년(15세?) 경 부친 강영숙을 따라 하와이로 ...",1945년 3월 23일 충청남도 논산에서 4남매 중 막내로 태어났다. 불교와 미신 ...,"샤를 1세 드 부르봉 (Charles Ier de Bourbon, 1401년 – 1...",각급 선거관리위원회 위원의 임기는 6년이며(114조 3항) 연임에 관해서는 제한이 ...,"1879년, 스위스의 수도 베른 교외의 뮌헨부흐제(Münchenbuchsee)에서 ...",주민등록이라 함은 행정기관이 관할구역 안에 거주하는 주민의 거주상황과 이동실태를 파...
1,mrc-0-004762,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,3호선 스카버러(Line 3 Scarborough) 또는 스카버러 RT (Scarb...,1878년에 개통된 사설 협궤 철도인 순스발-토르프스하마르 철도에서 시작한다. 18...,이 역이 교통 허브로 거듭나게 된 것은 거의 우연에 가까웠으며 기존에 스카버러 타운...,"1896년 1월에 ""보소 철도""가 소가 역과 오아미 역 사이에 철도를 놓은 것이 시...",이 역의 미래는 아직까지는 불투명하며 노후한 RT를 교체하기 위한 경전철 또는 중전...,1950년 10월 1일 슬루센-회카렝엔 구간이 개통되었다. 1957년 회토리에트-슬...,마운트조이역을 지나는 철도가 지어진 시점은 마컴 역이 지어진 시점은 1871년으로 ...,...,"라치오 철도(Ferrovie Laziali, 줄여서 FL) 혹은 로마 교외 철도는 ...","스페인의 교통은 여러 도로와 철도, 급속절철, 항공로, 항구 등을 포함하는 스페인 ...","법률적 의미로서의 궤도는 대한민국에서 과거 궤도사업법, 현재 이를 계승한 삭도·궤도...","라티나 역은 넓은 1층짜리 건물로, 화물 보관소와 인접한 곳에 위치해 있다. 승객 ...",싼충은 단수이 서안에 펼쳐진 대만의 중심지였고 맹갑(艋舺)이나 대도(大稲)의 대안에...,"쓰루미 선(鶴見線, つるみせん)은 일본의 철도 노선 중 하나이다. 동일본 여객철도가...",다카이시 시는 오사카 부의 센슈 지역의 북부(센보쿠)에 위치하고 북쪽과 동쪽은 사카...,"일둘브 막 카우산틴(Ildulb mac Causantín, ? - 962년) 침략왕...",수도인 비엔티안 인근에 왓타이 국제공항이 있다. 국내선 공항은 왓타이 국제공항을 ...,None
2,mrc-1-001810,촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,촌주의 연원은 고대 국가의 발전 과정에서 부족 사회의 크고 작은 촌락 집단을 이끌던...,촌락공동체(村落共同體)의 연구는 고대의 제도적 역사를 논하는 중요한 방식들 가운데 ...,"촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...","앨리스와 밥은 암호학에서 널리 쓰이지만, 나머지 것들은 그만큼 널리 쓰이지는 않는다...","원시공동사회에 있어서는 토지가 촌락에 의해 공동으로 소유되어 있었으나, 사유재산제의...","'노동-협동조합당'은 영국 선거위원회에 공통으로 등록된 명칭이며, 투표용지에도 후보...",플랙은 호주로 돌아와서 1898년에는 멜버른에 가족 회계 회사를 차렸으며 그가 주말...,1905년부터 그는 경성부에 설립 예정이던 대한도서관 설립을 위한 자금 모금과 부지...,...,에너지에 대한 접근성도 중요한 이슈다. 세계적으로 에너지에 대한 접근성은 평등하지 ...,브라운베스(Brown Bess)는 18세기 영국 육군의 제식 머스킷과 그 파생 무기...,연안이씨의 시조 이 무(李 茂) 의정공파 종중의 종중원으로서 1947년 4월 28일...,불을 가지고 노는 소녀는 리스벳이 1년동안의 세계여행을 끝내고 스웨덴으로 돌아오는 ...,"유럽의 마지막 빙하기는 기원전 130,000년 경에서부터 70,000년 경까지 지속...","이 지역에 인류가 정착한 것은 최소 기원전 4백만년전으로 추정되지만, 역사에 처음 ...",성매매문제에 포괄적으로 접근할 수 있는 법체계가 성매매법으로 도입된 것은 아니었다는...,뮐하우젠(Mühlhausen)은 독일 튀링겐주에 위치한 도시이다. 면적은 86.34...,포르투갈의 행정 구역은 본토의 18개 현(distrito)을 포함한 5개 지방(re...,None
3,mrc-1-000219,로타이르가 백조를 구하기 위해 사용한 것은?,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,백조가 배지로 널리 사용된 이유는 백조 기사 전설 때문이다. 이 이야기는 오늘날 라...,가스통 파리가 구분한 버전 II는 백조 기사 자신의 이야기가 포함된다. 이 이야기는...,"로엔그린\n13세기 초, 독일 시인 볼프람 폰 에셴바흐는 서사시 《파르치팔》에 백조...",리버리 배지\n\n던스터블 백조 장신구는 중세 시기 사용된 리버리 배지 가운데 가장...,;故 에드워드 뉴게이트(통칭 '흰 수염' 일 '시로히게')\n:성우 : 아리모토 킨...,〈로키의 말다툼〉은 로키가 다른 신들에게 악담을 퍼붓는 것이 주 내용이다. 로키는 ...,"백조는 유럽과 소련, 몽고, 중국 등지에서 번식하고 겨울에는 아프리카, 인도, 우리...",...,"미란 샤 (1366년 - 1408년 4월 16일?, 페르시아어: میران شاہ)...","1945년 1월 남은 기간 동안, 호엔슈타우펜은 독일 본토로 철수하기 위해 퇴각전을...",레기네는 1822년 1월 23일에 덴마크 코펜하겐의 프레데리크베르크 구역에서 태어났...,하르드라디 왕조(Hardrådeætta) 는 11세기에서 12세기 사이 북유럽의 강...,대흥동의 동명은 ‘동막하리’를 ‘대흥정’이라 하여 1936년 서울의 구역을 확장할 ...,소그디아나 (ə) 또는 소그디아는 고대 이란족 계열의 유목민족인 스키타이인들을 부르...,첼시는 창단 이래로 스탬퍼드 브리지 이외에 다른 경기장을 사용한 적이 없다. 스탬퍼...,펑후 제도는 군도 전체가 중화민국 타이완성의 펑후현에 속하고 있고 펑후섬(澎湖島)에...,None,None
4,mrc-1-000285,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,조직후원인식과 정서적 몰입사이의 관계는